In [2]:
import pandas as pd
import numpy as np
import os
import csv
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import ComplementNB
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
import time
import json
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import binarize





Using TensorFlow backend.


In [28]:
directory = "processed_paired_stimuli/"
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

#Dict = { } 


In [212]:
random_forest_results = {}

In [29]:
start_time = time.time()
count = 0
for filename in os.listdir(directory): 
    if filename.endswith(".csv"):
        filepath = directory + filename
        count = count+1
        file = pd.read_csv(filepath,sep=",",index_col=0)
        print(filepath)
        slide_1 , slide_2 = file['slideNumber'].unique()
        
        X = file.iloc[:, :-2]
        Y = file.iloc[:, -1:]
        Y = pd.DataFrame(binarize(Y, threshold=slide_1, copy=True))
        x_train, x_test,y_train ,y_test = train_test_split(X,Y,test_size=0.2, random_state=42)
        
        accuracy_rf , avg_accuracy_rf , conf_mat_rf , feature_imp = random_forest(X,Y, kfold)
        save_to_json(count , filename, accuracy_rf,avg_accuracy_rf,conf_mat_rf,feature_imp)
        
        #accuracy_nn , avg_accuracy_nn , conf_mat_nn = neural_network(X,Y, kfold)
        #accuracy_nb , avg_accuracy_nb , conf_mat_nb = naive_bays(X,Y,kfold)
        #accuracy_svm, avg_accuracy_svm, conf_mat_svm = svm_algorithm(X,Y,kfold)
        
        #Dict[count] = ({"filename":filename , "Neural_Network": 
         #             {"accuracy_results":accuracy_nn.tolist(),
          #             "mean_accuracy":avg_accuracy_nn, 
           #            "confusion_matrix": conf_mat_nn.tolist()
            #          } , 
             #     "Naive_Bayes": 
              #        {"accuracy_results":accuracy_nb.tolist(),
               #        "mean_accuracy":avg_accuracy_nb, 
                #       "confusion_matrix": conf_mat_nb.tolist()
                 #     },
                  #"svm":
                   #  {"accuracy_results":accuracy_svm.tolist(),
                    #   "mean_accuracy":avg_accuracy_svm, 
                     #  "confusion_matrix": conf_mat_svm.tolist()
                     #}
                 #})
        
#print("--- %s seconds ---" % (time.time() - start_time))
#with open("algorithm_data.json", 'w') as f:
    #json.dump(Dict, f)       
        

FileNotFoundError: [Errno 2] No such file or directory: '/disso/processed_paired_stimuli/'

In [231]:
with open("randomforest_results.json", 'w') as f:
    json.dump(random_forest_results, f)  

In [248]:
def neural_network(X,Y, kfold):
    
    neural_network = KerasClassifier(build_fn= createModel, 
                                 epochs=10, 
                                 batch_size=100, 
                                 verbose=0)
   
    results = cross_val_score(neural_network, X, np.ravel(Y), cv=kfold)
    avg_result = results.mean()
    
    y_pred = cross_val_predict(neural_network, X, np.ravel(Y), cv=k_fold) #training model using cross validation 
    conf_mat_nn = confusion_matrix(Y, y_pred)
    print(conf_mat_nn)
   
    return results , avg_result , conf_mat_nn
    
    



In [9]:
def createModel():
    model = Sequential()
    
    model.add(Dense(20, activation="relu", input_shape = (20,)))        
   
              
    model.add(Dense(5, activation= "relu"))

    
    model.add(Dense(1, activation= "sigmoid"))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    
    return model


In [198]:
def naive_bays(X,Y,kfold):
    
    clf = ComplementNB()# calls naive bayes algorithm
    clf.fit(X, np.ravel(Y))
    
    
  
    nb_accuracy = cross_val_score(clf, X, np.ravel(Y), cv=kfold, n_jobs=1)
    avg_accuracy = nb_accuracy.mean()
    y_pred = cross_val_predict(clf, X, np.ravel(Y), cv=kfold) #training model using cross validation 
    conf_mat_nb = confusion_matrix(Y, y_pred)
    
    return nb_accuracy , avg_accuracy , conf_mat_nb
    

In [199]:
def svm_algorithm(X,Y,kfold):

    svm = SVC()
    svm.set_params(gamma='auto').fit(X, np.ravel(Y))

    accuracy_svm = cross_val_score(svm, X, np.ravel(Y), cv=kfold, n_jobs=1)
    avg_accuracy = accuracy_svm.mean()
    y_pred_svm = cross_val_predict(svm, X, np.ravel(Y), cv=kfold) #training model using cross validation     
    conf_mat_svm = confusion_matrix(Y, y_pred_svm)
    
    return accuracy_svm, avg_accuracy, conf_mat_svm

In [209]:
def random_forest(X,Y,kfold):
    clf = RandomForestClassifier(n_estimators = 100)
    clf.fit(X,np.ravel(Y))
    
    accuracy_rf = cross_val_score(clf, X, np.ravel(Y), cv=kfold, n_jobs=1)
    avg_accuracy = accuracy_rf.mean()
    y_pred_rf = cross_val_predict(clf, X, np.ravel(Y), cv=kfold) #training model using cross validation     
    conf_mat_rf = confusion_matrix(Y, y_pred_rf)
    
    feature_imp = pd.Series(clf.feature_importances_,index=X.columns).sort_values(ascending=False)
    
    return accuracy_rf, avg_accuracy, conf_mat_rf, feature_imp

In [228]:
def save_to_json(count , filename, accuracy_rf,avg_accuracy_rf,conf_mat_rf,feature_imp): 
    random_forest_results[count] = ({"filename":filename , "Random_forest":
                      {"accuracy_results":accuracy_rf.tolist(),
                       "mean_accuracy":avg_accuracy_rf, 
                       "confusion_matrix": conf_mat_rf.tolist(),
                       "important_features":feature_imp.tolist()
                      } 
                 })


In [180]:
save_to_json('6_7.csv', k,3,conf_mat_nn,accuracy_nb,avg_accuracy_nb,conf_mat_nb,accuracy_svm,avg_accuracy_svm,conf_mat_svm)

In [187]:
Dict[2] = {"filename":"6_10.csv"}

In [177]:
k = np.asarray([1,2,3,4,5,6])

In [182]:
Dict.update({"filename":"6_9.csv"})

In [204]:
X.columns

Index(['Alpha_1', 'Alpha_2', 'Alpha_3', 'Alpha_4', 'Beta_1', 'Beta_2',
       'Beta_3', 'Beta_4', 'Delta_1', 'Delta_2', 'Delta_3', 'Delta_4',
       'Theta_1', 'Theta_2', 'Theta_3', 'Theta_4', 'Gamma_1', 'Gamma_2',
       'Gamma_3', 'Gamma_4'],
      dtype='object')

In [ ]:
{"filename": "6_7.csv", "Neural_Network": {"accuracy_results": [1, 2, 3, 4, 5, 6], "mean_accuracy": 3, "confusion_matrix": [[1449, 126], [254, 744]]}, "Naive_Bayes": {"accuracy_results": [0.8294573643410853, 0.7596899224806202, 0.7093023255813954, 0.7906976744186046, 0.7635658914728682, 0.7937743190661478, 0.8015564202334631, 0.7821011673151751, 0.703125, 0.8203125], "mean_accuracy": 0.775358258490936, "confusion_matrix": [[1178, 397], [181, 817]]}, "svm": {"accuracy_results": [0.9651162790697675, 0.9534883720930233, 0.9496124031007752, 0.9651162790697675, 0.9612403100775194, 0.9416342412451362, 0.9766536964980544, 0.9610894941634242, 0.9765625, 0.9609375], "mean_accuracy": 0.9611451075317469, "confusion_matrix": [[1552, 23], [77, 921]]}}

In [230]:
random_forest_results

{1: {'filename': '4_6.csv',
  'Random_forest': {'accuracy_results': [1.0,
    1.0,
    1.0,
    1.0,
    0.9965870307167235,
    0.9965753424657534,
    0.9965753424657534,
    1.0,
    0.9931506849315068,
    1.0],
   'mean_accuracy': 0.9982888400579737,
   'confusion_matrix': [[1345, 4], [4, 1571]],
   'important_features': [0.11696391594757277,
    0.10275918043767557,
    0.10126087169886573,
    0.07050334569602994,
    0.06528898558428527,
    0.0650797311928813,
    0.04646380131232049,
    0.04609366387243624,
    0.045073418104872524,
    0.044975039526246846,
    0.04389840028893762,
    0.03624552905213293,
    0.03498775249863054,
    0.03189451549618024,
    0.02866156883291348,
    0.02746226288096427,
    0.025802081046692817,
    0.024377095910676227,
    0.024055190737026232,
    0.01815364988265913]}},
 2: {'filename': '4_7.csv',
  'Random_forest': {'accuracy_results': [1.0,
    0.9915966386554622,
    1.0,
    1.0,
    0.9957983193277311,
    1.0,
    0.9957983193277

In [44]:

stimuli_dataset = "4_12.csv"

stimuli_data = pd.read_csv(stimuli_dataset,sep=",",index_col=0)
stimuli_1 , stimuli_2 = stimuli_data['slideNumber'].unique()
        
Xnew = stimuli_data.iloc[:, :-2]
Yn = stimuli_data.iloc[:, -1:]
Ynew = pd.DataFrame(binarize(Yn, threshold=stimuli_1, copy=True))

    
model = createModel()
model.fit(Xnew,Ynew,epochs=10,batch_size=100,verbose=0)
scores = model.evaluate(Xnew,Ynew, verbose =0)

model.save("model.h5")





In [46]:
import pickle

svm_new = SVC()
svm_new.set_params(gamma='auto').fit(Xnew, np.ravel(Ynew))

with open('svm_model.pkl','wb') as k:
    pickle.dump(svm_new,k)

https://stats.stackexchange.com/questions/52274/how-to-choose-a-predictive-model-after-k-fold-cross-validation - read through this when explaining why you chose the model you chose

In [11]:
scores

[0.25765521307148337, 0.932721734046936]

In [12]:
stimuli_1

4.0

In [49]:
import pickle

clf = RandomForestClassifier(n_estimators = 100)
clf.fit(Xnew,np.ravel(Ynew))

with open('rf_model.pkl','wb') as k:
    pickle.dump(clf,k)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [30]:
Ynew = pd.DataFrame(binarize(Ynew, threshold=stimuli_1, copy=False))

In [23]:
np.count_nonzero(Yn == 12.0) 

1267

In [43]:
type(Yn)

pandas.core.frame.DataFrame

In [48]:
Ynew

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
2611,1.0
2612,1.0
2613,1.0
2614,1.0


In [50]:
stimuli_1

4.0